In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns
import torch
from IPython.display import display

current_dir = os.getcwd()
PROJECT_ROOT = os.path.abspath(os.path.join(current_dir, "../../../../../"))

sys.path.append(os.path.join(PROJECT_ROOT, "shared_utilities"))
from metrics import *



In [2]:
ptid_list = [540, 544, 552, 559, 563, 567, 570, 575, 584, 588, 591, 596]

## **Dropout = 0.1**

In [3]:
dir_01 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/dropout/01")

dict_01 = {}

detailed_results_df_01 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_01, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_01[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_01, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_01 = pd.concat([detailed_results_df_01, detailed_df], axis=0)
    detailed_results_df_01.reset_index(drop=True, inplace=True)
detailed_results_df_01.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-1.222569,-1.422308,0.000000,0.000000,67.0,52.964188,0.0,0.000000,A,A,AP,hypo,540
1,0,1,-1.208338,-1.391253,0.002846,0.006211,68.0,55.146492,0.2,0.436460,A,A,AP,hypo,540
2,0,2,-1.194107,-1.394239,0.002846,-0.000597,69.0,54.936623,0.2,-0.041973,A,A,AP,hypo,540
3,0,3,-1.194107,-1.401631,0.000000,-0.001478,69.0,54.417168,0.0,-0.103890,A,A,AP,hypo,540
4,0,4,-1.165646,-1.407688,0.005692,-0.001211,71.0,53.991585,0.4,-0.085118,B,A,AP,eu,540


In [4]:
all_01_dfs = list(dict_01.values())

In [5]:
all_01_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_01_dfs]
aggregate_01_df = sum(all_01_value_only_dfs)
# add glycaemic region column
aggregate_01_df["Glycaemic Region"] = all_01_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_01_df = aggregate_01_df[["Glycaemic Region"] + list(aggregate_01_df.columns[:-1])]

aggregate_01_df["AP:EP"] = aggregate_01_df["AP"] / aggregate_01_df["EP"]

aggregate_01_df["AP%"] = aggregate_01_df["AP"] / aggregate_01_df["Count"] * 100
aggregate_01_df["BE%"] = aggregate_01_df["BE"] / aggregate_01_df["Count"] * 100
aggregate_01_df["EP%"] = aggregate_01_df["EP"] / aggregate_01_df["Count"] * 100

display(aggregate_01_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11678,355,1088,13121,10.733456,89.002363,2.705586,8.292051
1,eu,306830,16774,4034,327638,76.060982,93.649088,5.119675,1.231237
2,hyper,172829,9360,6252,188441,27.643794,91.715179,4.967072,3.317749
3,overall,491337,26489,11374,529200,43.198259,92.845238,5.005480,2.149282


In [6]:
for ptid, df in dict_01.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_01[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1971,22,220,2213,8.959091,89.064618,0.994126,9.941256
1,eu,29469,2184,123,31776,239.585366,92.739804,6.873112,0.387085
2,hyper,12736,1020,1047,14803,12.164279,86.036614,6.890495,7.072891
3,overall,44176,3226,1390,48792,31.781295,90.539433,6.611740,2.848828


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,135,0,25,160,5.400000,84.375000,0.000000,15.625000
1,eu,31078,1107,44,32229,706.318182,96.428682,3.434795,0.136523
2,hyper,15145,598,396,16139,38.244949,93.841006,3.705310,2.453684
3,overall,46358,1705,465,48528,99.694624,95.528355,3.513436,0.958210


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,84,1,11,96,7.636364,87.500000,1.041667,11.458333
1,eu,30097,1780,217,32094,138.695853,93.777653,5.546208,0.676139
2,hyper,25521,1584,801,27906,31.861423,91.453451,5.676199,2.870350
3,overall,55702,3365,1029,60096,54.132167,92.688365,5.599374,1.712260


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1824,103,194,2121,9.402062,85.997171,4.856200,9.146629
1,eu,34908,2622,1002,38532,34.838323,90.594830,6.804734,2.600436
2,hyper,14298,966,459,15723,31.150327,90.936844,6.143866,2.919290
3,overall,51030,3691,1655,56376,30.833837,90.517241,6.547112,2.935646


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1144,43,133,1320,8.601504,86.666667,3.257576,10.075758
1,eu,33446,1619,524,35589,63.828244,93.978476,4.549158,1.472365
2,hyper,12830,658,291,13779,44.089347,93.112708,4.775383,2.111909
3,overall,47420,2320,948,50688,50.021097,93.552715,4.577020,1.870265


## **Learning Rate = 0.3**

In [7]:


dir_03 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/dropout/03")

dict_03 = {}

detailed_results_df_03 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_03, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_03[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_03, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_03 = pd.concat([detailed_results_df_03, detailed_df], axis=0)
    detailed_results_df_03.reset_index(drop=True, inplace=True)
detailed_results_df_03.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-0.226418,-0.083371,0.000000,0.000000,137.0,147.05199,0.0,0.000000,A,A,AP,eu,540
1,0,1,-0.112572,0.021719,0.022769,0.021018,145.0,154.43670,1.6,1.476944,A,A,AP,eu,540
2,0,2,-0.027188,0.076729,0.017077,0.011002,151.0,158.30228,1.2,0.773113,A,A,AP,eu,540
3,0,3,0.029735,0.140938,0.011385,0.012842,155.0,162.81430,0.8,0.902406,A,A,AP,eu,540
4,0,4,0.072427,0.160736,0.008538,0.003960,158.0,164.20552,0.6,0.278244,A,A,AP,eu,540


In [8]:
all_03_dfs = list(dict_03.values())

In [9]:
all_03_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_03_dfs]
aggregate_03_df = sum(all_03_value_only_dfs)
# add glycaemic region column
aggregate_03_df["Glycaemic Region"] = all_03_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_03_df = aggregate_03_df[["Glycaemic Region"] + list(aggregate_03_df.columns[:-1])]

aggregate_03_df["AP:EP"] = aggregate_03_df["AP"] / aggregate_03_df["EP"]

aggregate_03_df["AP%"] = aggregate_03_df["AP"] / aggregate_03_df["Count"] * 100
aggregate_03_df["BE%"] = aggregate_03_df["BE"] / aggregate_03_df["Count"] * 100
aggregate_03_df["EP%"] = aggregate_03_df["EP"] / aggregate_03_df["Count"] * 100

display(aggregate_03_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11861,375,885,13121,13.402260,90.397073,2.858014,6.744913
1,eu,306928,16562,4148,327638,73.994214,93.678999,5.054969,1.266031
2,hyper,172447,9397,6597,188441,26.140215,91.512463,4.986707,3.500830
3,overall,491236,26334,11630,529200,42.238693,92.826153,4.976190,2.197657


In [10]:
for ptid, df in dict_03.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_03[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1983,28,202,2213,9.816832,89.606869,1.265251,9.127881
1,eu,29394,2228,154,31776,190.870130,92.503776,7.011581,0.484642
2,hyper,12756,976,1071,14803,11.910364,86.171722,6.593258,7.235020
3,overall,44133,3232,1427,48792,30.927120,90.451303,6.624037,2.924660


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,136,2,22,160,6.181818,85.000000,1.250000,13.750000
1,eu,31050,1125,54,32229,575.000000,96.341804,3.490645,0.167551
2,hyper,15120,600,419,16139,36.085919,93.686102,3.717702,2.596196
3,overall,46306,1727,495,48528,93.547475,95.421200,3.558770,1.020030


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,81,2,13,96,6.230769,84.375000,2.083333,13.541667
1,eu,30087,1787,220,32094,136.759091,93.746495,5.568019,0.685486
2,hyper,25490,1587,829,27906,30.747889,91.342364,5.686949,2.970687
3,overall,55658,3376,1062,60096,52.408663,92.615149,5.617678,1.767173


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1867,127,127,2121,14.700787,88.024517,5.987742,5.987742
1,eu,34968,2616,948,38532,36.886076,90.750545,6.789162,2.460293
2,hyper,14182,957,584,15723,24.284247,90.199071,6.086625,3.714304
3,overall,51017,3700,1659,56376,30.751658,90.494182,6.563076,2.942742


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1222,24,74,1320,16.513514,92.575758,1.818182,5.606061
1,eu,33452,1586,551,35589,60.711434,93.995336,4.456433,1.548231
2,hyper,12728,699,352,13779,36.159091,92.372451,5.072937,2.554612
3,overall,47402,2309,977,50688,48.517912,93.517203,4.555319,1.927478


## **Dropout = 0.2**

In [13]:
dir_02 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/batch_size/batch_size_256")

dict_02 = {}

detailed_results_df_02 = pd.DataFrame()

for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_02, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_02[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_02, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_02 = pd.concat([detailed_results_df_02, detailed_df], axis=0)
    detailed_results_df_02.reset_index(drop=True, inplace=True)
detailed_results_df_02.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,-0.012957,0.141815,0.000000,0.000000,152.0,162.87592,0.0,0.000000,A,A,AP,eu,540
1,0,1,-0.012957,0.198857,0.000000,0.011408,152.0,166.88428,0.0,0.801671,A,A,AP,eu,540
2,0,2,-0.069880,0.244570,-0.011385,0.009143,148.0,170.09656,-0.8,0.642456,A,B,AP,eu,540
3,0,3,-0.169495,0.251943,-0.019923,0.001475,141.0,170.61467,-1.4,0.103623,A,B,AP,eu,540
4,0,4,-0.269110,0.307721,-0.019923,0.011156,134.0,174.53420,-1.4,0.783904,B,lD,BE,eu,540


In [14]:
all_02_dfs = list(dict_02.values())

In [15]:
all_02_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_02_dfs]
aggregate_02_df = sum(all_02_value_only_dfs)
# add glycaemic region column
aggregate_02_df["Glycaemic Region"] = all_02_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_02_df = aggregate_02_df[["Glycaemic Region"] + list(aggregate_02_df.columns[:-1])]

aggregate_02_df["AP:EP"] = aggregate_02_df["AP"] / aggregate_02_df["EP"]

aggregate_02_df["AP%"] = aggregate_02_df["AP"] / aggregate_02_df["Count"] * 100
aggregate_02_df["BE%"] = aggregate_02_df["BE"] / aggregate_02_df["Count"] * 100
aggregate_02_df["EP%"] = aggregate_02_df["EP"] / aggregate_02_df["Count"] * 100

display(aggregate_02_df)

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11769,398,954,13121,12.336478,89.695907,3.033305,7.270787
1,eu,307219,16433,3986,327638,77.074511,93.767817,5.015596,1.216587
2,hyper,172366,9448,6627,188441,26.009657,91.469479,5.013771,3.516751
3,overall,491354,26279,11567,529200,42.478949,92.848450,4.965797,2.185752


In [16]:
for ptid, df in dict_02.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_02[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1941,32,240,2213,8.087500,87.708992,1.446001,10.845007
1,eu,29492,2154,130,31776,226.861538,92.812185,6.778701,0.409114
2,hyper,12698,1056,1049,14803,12.104862,85.779909,7.133689,7.086401
3,overall,44131,3242,1419,48792,31.100070,90.447204,6.644532,2.908264


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,129,2,29,160,4.448276,80.625000,1.250000,18.125000
1,eu,31127,1061,41,32229,759.195122,96.580719,3.292066,0.127215
2,hyper,15103,610,426,16139,35.453052,93.580767,3.779664,2.639569
3,overall,46359,1673,496,48528,93.465726,95.530415,3.447494,1.022090


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,82,3,11,96,7.454545,85.416667,3.125000,11.458333
1,eu,30096,1788,210,32094,143.314286,93.774537,5.571135,0.654328
2,hyper,25522,1526,858,27906,29.745921,91.457034,5.468358,3.074608
3,overall,55700,3317,1079,60096,51.621872,92.685037,5.519502,1.795461


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1866,144,111,2121,16.810811,87.977369,6.789250,5.233380
1,eu,35108,2436,988,38532,35.534413,91.113879,6.322018,2.564103
2,hyper,14105,1042,576,15723,24.487847,89.709343,6.627234,3.663423
3,overall,51079,3622,1675,56376,30.494925,90.604158,6.424720,2.971122


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1172,52,96,1320,12.208333,88.787879,3.939394,7.272727
1,eu,33396,1650,543,35589,61.502762,93.837984,4.636264,1.525752
2,hyper,12794,642,343,13779,37.300292,92.851441,4.659264,2.489295
3,overall,47362,2344,982,50688,48.230143,93.438289,4.624369,1.937342


## **Learning Rate = 0.4**

In [18]:
dir_04 = os.path.join(PROJECT_ROOT, "models/jpformer/fine_tuning_development_files/dropout/04")
dict_04 = {}
detailed_results_df_04 = pd.DataFrame()
for ptid in ptid_list:
    df = pd.read_csv(os.path.join(dir_04, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_overall_cg_ega.csv"))
    df.rename(columns={"Unnamed: 0": "Glycaemic Region"}, inplace=True)
    dict_04[ptid] = df

    detailed_df = pd.read_csv(os.path.join(dir_04, f"patient_{ptid}/fine_tuning_eval/patient_{ptid}_detailed_test_results.csv"))
    detailed_df['ptid'] = ptid
    detailed_results_df_04 = pd.concat([detailed_results_df_04, detailed_df], axis=0)
    detailed_results_df_04.reset_index(drop=True, inplace=True)
detailed_results_df_04.head()

,sequence_id,timepoint,true,predicted,dy_true,dy_pred,true_glucose,predicted_glucose,dy_true_glucose,dy_pred_glucose,P_EGA_Class,R_EGA_Class,CG_EGA_Class,glycemic_region,ptid
0,0,0,0.172042,0.326601,0.000000,0.000000,165.0,175.86095,0.0,0.000000,A,A,AP,eu,540
1,0,1,0.157811,0.372804,-0.002846,0.009241,164.0,179.10765,-0.2,0.649340,A,A,AP,eu,540
2,0,2,0.115119,0.409332,-0.008538,0.007306,161.0,181.67447,-0.6,0.513363,A,B,AP,eu,540
3,0,3,0.129350,0.447038,0.002846,0.007541,162.0,184.32411,0.2,0.529930,A,A,AP,eu,540
4,0,4,0.043966,0.456988,-0.017077,0.001990,156.0,185.02328,-1.2,0.139834,A,B,AP,eu,540


In [19]:
all_04_dfs = list(dict_04.values())

In [20]:
all_04_value_only_dfs = [df.drop(columns=["Glycaemic Region"]) for df in all_04_dfs]
aggregate_04_df = sum(all_04_value_only_dfs)
# add glycaemic region column
aggregate_04_df["Glycaemic Region"] = all_04_dfs[0]["Glycaemic Region"]
# move glycaemic region to the front
aggregate_04_df = aggregate_04_df[["Glycaemic Region"] + list(aggregate_04_df.columns[:-1])]
aggregate_04_df["AP:EP"] = aggregate_04_df["AP"] / aggregate_04_df["EP"]
aggregate_04_df["AP%"] = aggregate_04_df["AP"] / aggregate_04_df["Count"] * 100
aggregate_04_df["BE%"] = aggregate_04_df["BE"] / aggregate_04_df["Count"] * 100
aggregate_04_df["EP%"] = aggregate_04_df["EP"] / aggregate_04_df["Count"] * 100
display(aggregate_04_df)    

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11799,376,946,13121,12.472516,89.924548,2.865635,7.209816
1,eu,306993,16504,4141,327638,74.134992,93.698838,5.037267,1.263895
2,hyper,172475,9414,6552,188441,26.324023,91.527322,4.995728,3.476950
3,overall,491267,26294,11639,529200,42.208695,92.832011,4.968632,2.199358


In [21]:
for ptid, df in dict_04.items():
    df["AP:EP"] = df["AP"] / df["EP"]
    df["AP%"] = df["AP"] / df["Count"] * 100
    df["BE%"] = df["BE"] / df["Count"] * 100
    df["EP%"] = df["EP"] / df["Count"] * 100
    dict_04[ptid] = df  # Update the dictionary with the modified DataFrame
    print(f"Patient ID: {ptid}")
    display(df)

Patient ID: 540


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,2002,24,187,2213,10.705882,90.465432,1.084501,8.450068
1,eu,29435,2171,170,31776,173.147059,92.632805,6.832200,0.534995
2,hyper,12708,1013,1082,14803,11.744917,85.847463,6.843207,7.309329
3,overall,44145,3208,1439,48792,30.677554,90.475898,6.574848,2.949254


Patient ID: 544


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,141,2,17,160,8.294118,88.125000,1.250000,10.625000
1,eu,31069,1094,66,32229,470.742424,96.400757,3.394458,0.204785
2,hyper,15101,651,387,16139,39.020672,93.568375,4.033707,2.397918
3,overall,46311,1747,470,48528,98.534043,95.431503,3.599984,0.968513


Patient ID: 552


...

,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,87,2,7,96,12.428571,90.625000,2.083333,7.291667
1,eu,30111,1768,215,32094,140.051163,93.821275,5.508818,0.669907
2,hyper,25508,1558,840,27906,30.366667,91.406866,5.583029,3.010105
3,overall,55706,3328,1062,60096,52.453861,92.695021,5.537806,1.767173


Patient ID: 591


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1842,121,158,2121,11.658228,86.845827,5.704856,7.449316
1,eu,34938,2550,1044,38532,33.465517,90.672688,6.617876,2.709436
2,hyper,14260,995,468,15723,30.470085,90.695160,6.328309,2.976531
3,overall,51040,3666,1670,56376,30.562874,90.534979,6.502767,2.962253


Patient ID: 596


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,1147,51,122,1320,9.401639,86.893939,3.863636,9.242424
1,eu,33484,1604,501,35589,66.834331,94.085251,4.507011,1.407738
2,hyper,12776,639,364,13779,35.098901,92.720807,4.637492,2.641701
3,overall,47407,2294,987,50688,48.031408,93.527068,4.525726,1.947206


## **Learning Rate Comparison**

In [22]:
display(aggregate_01_df)

display(aggregate_02_df)

display(aggregate_03_df)

display(aggregate_04_df)



,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11678,355,1088,13121,10.733456,89.002363,2.705586,8.292051
1,eu,306830,16774,4034,327638,76.060982,93.649088,5.119675,1.231237
2,hyper,172829,9360,6252,188441,27.643794,91.715179,4.967072,3.317749
3,overall,491337,26489,11374,529200,43.198259,92.845238,5.005480,2.149282


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11769,398,954,13121,12.336478,89.695907,3.033305,7.270787
1,eu,307219,16433,3986,327638,77.074511,93.767817,5.015596,1.216587
2,hyper,172366,9448,6627,188441,26.009657,91.469479,5.013771,3.516751
3,overall,491354,26279,11567,529200,42.478949,92.848450,4.965797,2.185752


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11861,375,885,13121,13.402260,90.397073,2.858014,6.744913
1,eu,306928,16562,4148,327638,73.994214,93.678999,5.054969,1.266031
2,hyper,172447,9397,6597,188441,26.140215,91.512463,4.986707,3.500830
3,overall,491236,26334,11630,529200,42.238693,92.826153,4.976190,2.197657


,Glycaemic Region,AP,BE,EP,Count,AP:EP,AP%,BE%,EP%
0,hypo,11799,376,946,13121,12.472516,89.924548,2.865635,7.209816
1,eu,306993,16504,4141,327638,74.134992,93.698838,5.037267,1.263895
2,hyper,172475,9414,6552,188441,26.324023,91.527322,4.995728,3.476950
3,overall,491267,26294,11639,529200,42.208695,92.832011,4.968632,2.199358


In [23]:
results_list = [aggregate_01_df, aggregate_02_df, aggregate_03_df, aggregate_04_df]
results_dict = {}

for i, table in enumerate(results_list):
    #delete last row
    table = table[:-1]
    average_ap_ep = table["AP:EP"].mean()
    results_dict[f"Table {i+1}"] = average_ap_ep

for key, value in results_dict.items():
    print(f"{key}: {value}")

Table 1: 38.146077174734025
Table 2: 38.47354874569228
Table 3: 37.84556307181189
Table 4: 37.64384353439842


In [24]:
best_hypo_01 = []
best_hypo_02 = []
best_hypo_03 = []


for ptid in ptid_list:

    hypo_dict = {}
    hypo_01 = dict_01[ptid].loc[dict_01[ptid]["Glycaemic Region"] == "hypo"]
    hypo_01_apep = hypo_01["AP:EP"].values[0]
    hypo_dict["01"] = hypo_01_apep

    hypo_02 = dict_02[ptid].loc[dict_02[ptid]["Glycaemic Region"] == "hypo"]
    hypo_02_apep = hypo_02["AP:EP"].values[0]
    hypo_dict["02"] = hypo_02_apep

    hypo_03 = dict_03[ptid].loc[dict_03[ptid]["Glycaemic Region"] == "hypo"]
    hypo_03_apep = hypo_03["AP:EP"].values[0]
    hypo_dict["03"] = hypo_03_apep

    best_hypo = max(hypo_dict, key=hypo_dict.get)
    if best_hypo == "01":
        best_hypo_01.append(ptid)
    elif best_hypo == "02":
        best_hypo_02.append(ptid)
    elif best_hypo == "03":
        best_hypo_03.append(ptid)
    else:
        print("Error: No best hypo found for patient ID:", ptid)

print("Dropout 0.1 results in best Hypo AP:EP for ptid: ", best_hypo_01)
print("Dropout 0.2 results in best Hypo AP:EP for ptid: ", best_hypo_02)
print("Dropout 0.3 results in best Hypo AP:EP for ptid: ", best_hypo_03)



Dropout 0.1 results in best Hypo AP:EP for ptid:  [584, 588]
Dropout 0.2 results in best Hypo AP:EP for ptid:  [552, 559, 570, 591]
Dropout 0.3 results in best Hypo AP:EP for ptid:  [540, 544, 563, 567, 575, 596]


In [25]:
best_overall_01 = []
best_overall_02 = []
best_overall_03 = []

for ptid in ptid_list:

    overall_dict = {}
    overall_01 = dict_01[ptid].loc[dict_01[ptid]["Glycaemic Region"] == "overall"]
    overall_01_apep = overall_01["AP:EP"].values[0]
    overall_dict["01"] = overall_01_apep

    overall_02 = dict_02[ptid].loc[dict_02[ptid]["Glycaemic Region"] == "overall"]
    overall_02_apep = overall_02["AP:EP"].values[0]
    overall_dict["02"] = overall_02_apep

    overall_03 = dict_03[ptid].loc[dict_03[ptid]["Glycaemic Region"] == "overall"]
    overall_03_apep = overall_03["AP:EP"].values[0]
    overall_dict["03"] = overall_03_apep

    best_overall = max(overall_dict, key=overall_dict.get)
    if best_overall == "01":
        best_overall_01.append(ptid)
    elif best_overall == "02":
        best_overall_02.append(ptid)
    elif best_overall == "03":
        best_overall_03.append(ptid)
    else:
        print("Error: No best overall found for patient ID:", ptid)

print("Dropout 0.1 results in best Overall AP:EP for ptid: ", best_overall_01)
print("Dropout 0.2 results in best Overall AP:EP for ptid: ", best_overall_02)
print("Dropout 0.3 results in best Overall AP:EP for ptid: ", best_overall_03)


Dropout 0.1 results in best Overall AP:EP for ptid:  [540, 544, 552, 559, 563, 584, 588, 591, 596]
Dropout 0.2 results in best Overall AP:EP for ptid:  [567, 575]
Dropout 0.3 results in best Overall AP:EP for ptid:  [570]
